# Exploring deep sea acoustic events
## Whale song detector: CNN estimator model

### Feb 2020 PDSG Applied Data Science Meetup series<br>John Burt

### Session details

For February’s four session meetup series we’ll be working with long term hydrophone recordings from University of Hawaii's Aloha Cabled Observatory (ACO - http://aco-ssds.soest.hawaii.edu), located at a depth of 4728m off Oahu. The recordings span a year and contain many acoustic events: wave movements, the sound of rain, ship noise, possible bomb noises, geologic activity and whale calls and songs. There is a wide range of project topics to explore: identifying and counting acoustic events such as whale calls, measuring daily or seasonal noise trends, measuring wave hydrodynamics, etc.

### This notebook:

I built a classifier model to detect whale vocalizations in the recording. For this I used a standard Tensorflow/Keras Convolutional Neural Network (CNN), with a sound spectrograph as input.

The CNN model was trained using a generator function that combined background sounds from the hydrophone recording with whale vocalization clips selected from clean song recordings acquired from the Woods Hole Oceanographic Institution's Watkins Marine Mammal Sound Database. The generator function randomly combined noise and whale sounds so that each sample was unique.

In this notebook, I take the following steps:

- Build a sklearn CNN estimator model to classify audio clips as having whale song or not.
- Train model using a generator function that creates unique samples (whale or no-whale).
- Test model using a test set of whale sounds held out from training.
- Scan a hydrophone recording with the model, save whale detections as audio clips

Extra packages required:
- librosa


In [14]:
# remove warnings
import warnings
warnings.filterwarnings('ignore')
# ---

%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

import pandas as pd
pd.options.display.max_columns = 100

import numpy as np

import librosa
import librosa.display

import soundfile as sf
import bz2
import pickle

## Define the whale song detection model

The model is defined as an sklearn estimator object. That allows me to easily run sklearn tools like kfolds cross validation and hyperparameter tuning.

The network model I've chosen is a standard tensorflow/keras based CNN, commonly used to classify images. Model input is a audio spectrograph (a time x frequency representation of a sound). Spectrographs are equivalent to images, so a CNN model should be a good first choice.

### The model layers:

- Input (Spectrograph image)
- Conv2D
- MaxPooling2D
- Conv2D
- MaxPooling2D
- Conv2D
- MaxPooling2D
- Conv2D
- MaxPooling2D
- Flatten
- Dense
- Output (no whale song, whale song present)

### Model notes:

- During testing I tried using BatchNormalization layers after each Conv2D. This yielded poor results, where often the model would never learn. Therefore BatchNormalization was omitted. 


- I also tried a Dropout layer after each Conv2D. These networks learned, but not faster or better than omitting Dropout. Therefore I've disabled Dropout.


- The model uses a sample generator for training and testing. The generator combines background hydrophone noise with clean humpback whale song note clips to produce a simulation of whale song with the same background the detector will be dealing with in the recording. More details:
    - Background sound was randomly sampled from the hydrophone recording and totalled about 9 hours of audio. I manually verified that the background audio had no whale song.
    
    - Target whale sounds were clipped from WHOI recordings. They ranged from 0.5 to 5 seconds long.
    
    - Both background sound and target sounds were scaled to +/- 1 amplitude. For generating training samples, target whale sounds were randomly attenuated before adding to background noise, to simulate the range of song amplitudes in the hydrophone recording.
        
    - A model input frame was created by randomly selecting a section of background noise, randomly selecting a target sound and adding the target to the noise at a random time point within the noise frame, attenuating the target to a random degree. The noise+whale waveform was converted into a spectrograph for model input.
        
    - Negative (no whale) samples were generated exactly like positive (whale present), except no target was added to the background.
        
    - For testing, a set of target notes were held out from training (same background was used for training and testing). The generator was then used to create a test set for validation. 

In [16]:
from sklearn.base import BaseEstimator, ClassifierMixin
from keras.utils import to_categorical
# import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD

tf.logging.set_verbosity(tf.logging.ERROR)

from sklearn.metrics import accuracy_score

# sklearn estimator model format for sound detector
class SoundDetector(BaseEstimator, ClassifierMixin):
    """recommender engine as an estimator"""

    def __init__(self, 
        samprate = 5000,
        hoplength = None, # # samples between FFTs
        fftsize = 512, # FFT size (#bins = fftsize/2)
        framesize = 1000, # frame size, samples
        targwtrange = [.05, .1], # test: 88% no white, 84% white
        whitenpct = None, # whitening percentile (more==more whitening)
        freqrange = [60,4000], # frequency range to keep
        sampjitter = 2000, # time jitter range, samples
        ptarget=0.5, # prob target sample in generated data
        ):      
        
        """
        Called when initializing the model
        """
        # model parameters
        self.samprate = samprate
        self.hoplength = hoplength
        self.fftsize = fftsize
        self.framesize = framesize
        self.targwtrange = targwtrange
        self.whitenpct = whitenpct
        self.freqrange = freqrange
        self.sampjitter = sampjitter
        self.ptarget = ptarget 
        
        self.CNN_model = None
        self.train_targets = None
        self.test_targets = None

    # ******************************************************************
    def set_params(self, **params):
        self.__dict__.update(params)
        
    # ******************************************************************
    def whiten_spec(self, spec, pctile=95):
        """Whiten the spectrograph by thresholding using the given percentile (0-100)"""
        specw = spec.copy()
        for row in range(spec.shape[0]):
            specw[row,specw[row,:]<np.percentile(specw[row,:],pctile)] = 0
        return specw
    
    # ******************************************************************
    def make_spec(self, wave):
        """Generate a spectrograph of waveform data using the model parameters"""
        # return whitened spec
        if self.whitenpct is None:
            return np.abs(librosa.stft(wave, hop_length=self.hoplength, n_fft=self.fftsize))
        # return spec as-is
        else:
            return whiten_spec(np.abs(librosa.stft(wave, hop_length=self.hoplength, 
                                                   n_fft=self.fftsize)), whitenpct)

    # ******************************************************************
    def make_sample_spec(self, target, background, framesize,
                          targwtrange=[1,1], whitenpct=None, hoplength=0, n_fft=256, 
                          samprate=4000, freqrange = [0,-1], sampjitter=0):
        """generate a sample spectrograph with target plus background noise"""

        # add random jitter to where target sound starts in sample frame 
        jitter = int(np.random.random() * sampjitter*2 - sampjitter)
        offset = max(0,min(framesize-len(target),jitter+int((framesize-len(target))/2)))

        # weight the target, either fixed, or over random range
        if targwtrange[1] == targwtrange[0]:
            targetwt = targwtrange[1]
        else:
            targetwt = np.random.random() * (targwtrange[1]-targwtrange[0]) +  targwtrange[0]

        # random select where frame starts in background noise 
        backstart = int((len(background) - framesize) * np.random.random())

        # create frame of framesize length containing target, buffering as necessary
        frame = np.zeros([framesize,]) 
        targetlen = min(len(target),framesize)
        frame[offset:offset+targetlen] = target[:targetlen]*targetwt

        # determine upper and lower frequency range bin indices 
        nbins = int(n_fft/2)
        minbin = int(np.round(freqrange[0]*nbins*2/samprate))
        if freqrange[1] > 0:
            maxbin = min(nbins,int(np.round(freqrange[1]*nbins*2/samprate)))
        else:
            maxbin = nbins
            
        return self.make_spec(frame + background[backstart:backstart+framesize])

    # ******************************************************************
    def sample_generator(self, target, background, batch_size, framesize, targwtrange=[1,1], whitenpct=None, 
                         hoplength=0, n_fft=256, samprate=4000, freqrange = [0,-1], sampjitter=0, ptarget=0.5):
        """Generator to create batches of model training spectrograph samples.
        For each sample, randomly combines a target sound with a background."""

        # create a sample frame to get spec dimensions. 
        tspec = self.make_sample_spec(target['wave'].iloc[0], background['wave'].iloc[0],
                                 framesize, targwtrange=targwtrange, 
                                 whitenpct=whitenpct, freqrange=freqrange, hoplength=hoplength, 
                                 n_fft=fftsize, sampjitter=sampjitter)
        # spec image dimensions
        spec_x = tspec.shape[0]
        spec_y = tspec.shape[1]
        numchans = 1

        # Create empty arrays to contain batch of X (features) and y (labels)
        batch_X = np.zeros((batch_size, spec_x, spec_y, numchans))
        batch_y = np.zeros((batch_size,))

        # yield a batch of novel sample spectrograph images, 
        # randomly selected as containing whale or no-whale: 
        #  no-whale sample: background with no whale.
        #  whale: background w/ whale song note added at random time, w/ random range of amplitude.
        while True:
            # random select target and background file path indices
            id_idx = np.random.choice(range(target.shape[0]), size=batch_size)
            back_idx = np.random.choice(range(background.shape[0]), size=batch_size)
            speclist = []
            y = []
            for i, tid, bid in zip(range(len(id_idx)), id_idx, back_idx):
                # is this a whale (target) or no-whale sample?
                istarget = 1 if np.random.random() < ptarget else 0
                batch_y[i] = istarget
                # generate a spectrograph image and append to batch list
                speclist.append(
                    self.make_sample_spec(target['wave'].iloc[tid], background['wave'].iloc[bid],
                        framesize, targwtrange=targwtrange if istarget else [0,0], 
                        whitenpct=whitenpct, freqrange=freqrange, hoplength=hoplength,
                        n_fft=fftsize, sampjitter=sampjitter)            
                        )
            # convert list of 2D images to 3D array, rearrange axes
            specs = np.moveaxis(np.dstack(speclist),[0,1,2], [1,2,0] )
            # reshape array to X[sample#, image_width, image_height]
            batch_X = specs.reshape(specs.shape[0], spec_x, spec_y, 1 )

            # note that for now, I'm assuming only two classes: whale vs no-whale
            yield batch_X, to_categorical(batch_y,num_classes=2)
     
    # ******************************************************************
    def train_test_target_split(self, targets, testp=0.1):
        """Divide target sounds into train or test categories.
        
        This is needed to prevent info leakage into test set.
        testp = proportion of target sounds used for testing.
        Note: background sound is shared across train/test sets."""
        
        # randomly select train and test IDs
        target_ids = targets['id'].unique()
        np.random.shuffle(target_ids)
        n_testids = int(len(target_ids)*testp)
        train_ids = target_ids[:-n_testids]
        test_ids = target_ids[-n_testids:]

        train_targets = targets[ [tid in train_ids for tid in targets['id']] ]
        test_targets = targets[ [tid in test_ids for tid in targets['id']] ]

        return train_targets, test_targets

    # ******************************************************************
    def create_validation_set(self, targets, backgrounds, num_samples):
        """Yield a single batch of samples to use as a validation set."""
        for X_test, y_test in self.sample_generator(targets, backgrounds, num_samples, 
                             self.framesize, targwtrange=self.targwtrange, whitenpct=self.whitenpct, 
                             hoplength=self.hoplength, n_fft=self.fftsize, samprate=self.samprate, 
                             freqrange=self.freqrange, sampjitter=self.sampjitter, ptarget=self.ptarget
                            ):
            break
        return X_test, y_test
    
    # ******************************************************************
    def create_network(self, input_shape):
        """Build and compile keras CNN model"""
        
        # params for each conv layer: #filters, #strides, pooling_size
        convlayersize = [(2,7,(2,2),(2,2)), 
                         (4,3,(2,2),(2,2)),
                         (8,3,(2,2),(2,2)),
                         (16,3,(2,2),(2,2))
                        ]
        denselayersize = 100 # nodes in dense layer between C layers and output
        num_classes = 2 # #output nodes
        dropoutrate = 0 # dropout rate: 0 = no dropouts
        usebatchnorm = False # flag to use batch normalization

        model = Sequential()
        firstlayer = True           
        for (nfilters, kernelsize, strides, poolingsize) in convlayersize:
            # first conv filter layer connects to input
            if firstlayer:
                firstlayer = False
                model.add(Conv2D(nfilters, kernel_size=kernelsize, 
                                 input_shape=input_shape, 
                                 use_bias=False if usebatchnorm else True))
            # hidden conv filter layers
            else:
                model.add(Conv2D(nfilters, kernel_size=kernelsize, 
                                 use_bias=False if usebatchnorm else True))
            model.add(Activation("relu"))
            if dropoutrate > 0: model.add(Dropout(dropoutrate))
            if usebatchnorm: model.add(BatchNormalization())
            model.add(MaxPooling2D(pool_size=poolingsize, strides=strides))

        # dense post-CNN layer 
        model.add(Flatten())
        model.add(Dense(denselayersize, activation='relu'))
        
        # output layer
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(loss='categorical_crossentropy',
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=['accuracy'])   
        return model
        
    # ******************************************************************
    def fit(self, train_targets, backgrounds, test_targets=None):
        """ Train the model.
        """
        num_validation_samples = 250
        
        # split targets into training and validation sets
        if test_targets is not None:
            X_test, y_test = self.create_validation_set(test_targets, backgrounds, num_validation_samples)
        else:
            X_test, y_test = self.create_validation_set(train_targets, backgrounds, 1)

        # use test samples to get input shape for model training
        input_shape = (X_test.shape[1], X_test.shape[2], 1)

        # create the CNN model
        self.CNN_model = self.create_network(input_shape)
        
        batch_size = 10
        steps = batch_size*4
        epochs = 15
        updaterate = 5

        # train model using sample generator method. 
        # This allows unique training samples to be generated every batch.
        
        # Train model using a validation set
        if test_targets is not None:
            self.CNN_model.fit_generator(
                self.sample_generator(train_targets, backgrounds, batch_size, 
                                 self.framesize, targwtrange=self.targwtrange, whitenpct=self.whitenpct, 
                                 hoplength=self.hoplength, n_fft=self.fftsize, samprate=self.samprate, freqrange=self.freqrange, 
                                 sampjitter=self.sampjitter, ptarget=self.ptarget),
                steps_per_epoch = steps,
                epochs = epochs,
                validation_data = (X_test, y_test),
                verbose = 0
                )
        # Train model with no validation set
        else:
            self.CNN_model.fit_generator(
                self.sample_generator(train_targets, backgrounds, batch_size, 
                                 self.framesize, targwtrange=self.targwtrange, whitenpct=self.whitenpct, 
                                 hoplength=self.hoplength, n_fft=self.fftsize, samprate=self.samprate, freqrange=self.freqrange, 
                                 sampjitter=self.sampjitter, ptarget=self.ptarget),
                steps_per_epoch = steps,
                epochs = epochs,
                verbose = 0
                )
        return self
        
    # ******************************************************************
    def predict(self, specframes):
        """predict whether a set of spectrographs formatted for 
        model input contains a target sound. Returns index of max output. 
        """
        # return index of highest level output (0 or 1)
        return np.argmax(self.CNN_model.predict(specframes),axis=1) 

    # ******************************************************************
    def predict_proba(self, specframes):
        """predict whether a set of spectrographs formatted for 
        model input contains a target sound. Returns model outputs.
        """
        # treat model output as probability
        return self.CNN_model.predict(specframes)                           

    # ******************************************************************
    def predict_from_wave(self, wave):
        """Predict whether a waveform contains target sound.
        Returns index of max output.
        This is used by the whale detector script."""
        numframes = int(wave.shape[0]/self.framesize)
        speclist=[]
        for fnum in range(numframes):
            speclist.append(self.make_spec(wave[fnum*self.framesize:(fnum+1)*self.framesize]))
        spec_x = speclist[0].shape[0]
        spec_y = speclist[0].shape[1]
        specs = np.moveaxis(np.dstack(speclist),[0,1,2], [1,2,0] )
        X = specs.reshape(specs.shape[0], spec_x, spec_y, 1 )
        pred = self.CNN_model.predict(X)
        return np.argmax(pred,axis=1) 

    # ******************************************************************
    def score(self, y_true, y_pred):
        """mean percent of y_true game IDs in y_pred"""
        return  accuracy_score(y_true, y_pred)
          

## Load the train / test data

The training audio consists of clean clips of whale song notes (training targets), and background sounds with no whale song that I randomly selected from the hydrophone recordings. In another notebook, I prepared the audio by resampling the targets and background noise to the same sample rate (5 kHz) and saved them to pandas dataframes in HDF5 format. This allows the training audio to be quickly loaded, ready to be used with the model.

In [17]:
%%time

datapath = './data/model/wavdata_v1.h5'

targets = pd.read_hdf(datapath, key='target')
backgrounds = pd.read_hdf(datapath, key='background')

Wall time: 4.45 s


In [19]:
# define model params and a function to return an initialized model instance

batch_size = 250 # #training samples per batch

samprate = 5000 # audio sample rate
fftsize = 512 # FFT size (#bins = fftsize/2)
hoplength = None # #samples between FFTs (None = 1/4 fftsize)
framesize = samprate * 4 # frame size, samples
targwtrange = [.1, .2] # test: 93% no white, 95% white
whitenpct = None # whitening percentile (more==more whitening)
freqrange = [60,4000] # frequency range to keep
sampjitter = int(framesize/2) # time jitter range, samples
ptarget=0.5 # prob target sample in generated data

def make_clf():
    return SoundDetector(
        samprate = samprate,
        hoplength = hoplength, # # samples between FFTs
        fftsize = fftsize, # FFT size (#bins = fftsize/2)
        framesize = framesize, # frame size, samples
        targwtrange = targwtrange, # test: 88% no white, 84% white
        whitenpct = whitenpct, # whitening percentile (more==more whitening)
        freqrange = freqrange, # frequency range to keep
        sampjitter = sampjitter, # time jitter range, samples
        ptarget=ptarget, # prob target sample in generated data
        )

## Cross validate the model

Note that there are a number of parameters that define how the model creates input for training. I've manually experimented with these to find parameters that work pretty well. In the future, I'll tune these properly using hyperopt.

In [18]:
%%time
from sklearn.model_selection import KFold


kf = KFold(n_splits=5, shuffle=True)
scores = []
for train_index, test_index in kf.split(targets):
    clf = make_clf() # create new model instance
    # select train and test target waves
    train_targets = targets.iloc[train_index,:]
    test_targets = targets.iloc[test_index,:]
    # train the model
    clf.fit(train_targets, backgrounds, test_targets=test_targets)
    # create a validation spectrograph set from the test target waves
    X_test, y_test = clf.create_validation_set(test_targets, backgrounds, 500)
    # predict target sound presence
    y_pred = clf.predict(X_test)
    # generate a score
    scores.append(clf.score(np.argmax(y_test,axis=1), y_pred))
    print('score=',scores[-1])
    
print('mean score = %1.3f'%(np.mean(scores)))



250/250 [==============================] - 3s 13ms/sample - loss: 0.1164 - acc: 0.9560
score= 0.956
250/250 [==============================] - 3s 13ms/sample - loss: 0.1463 - acc: 0.9640
score= 0.952
250/250 [==============================] - 3s 13ms/sample - loss: 0.1773 - acc: 0.9320
score= 0.942
250/250 [==============================] - 4s 15ms/sample - loss: 0.1437 - acc: 0.9320
score= 0.948
250/250 [==============================] - 4s 15ms/sample - loss: 0.1591 - acc: 0.9360
score= 0.938
mean score = 0.947
Wall time: 15min 58s


## Train the model on all targets



In [20]:
%%time

# train the model
clf.fit(targets, backgrounds, test_targets=None)


Wall time: 2min 14s


SoundDetector(fftsize=512, framesize=20000, freqrange=[60, 4000],
              hoplength=None, ptarget=0.5, sampjitter=10000, samprate=5000,
              targwtrange=[0.1, 0.2], whitenpct=None)

## Scan a recording and predict whale song

Now that the model is trained, it's time to test it out on a real hydrophone recording.

For this task, I'll scan the "every_other_hour" recording that spans all of 2015, with 5 minutes of audio sampled every other hour. The script saves the audio for any frames the model classifies as having whale song to a detections folder for later review. 

In [68]:
import os
import fnmatch
from datetime import datetime,timedelta  

def read_clipnames(srcdir):
    """Walk through base folder and collect paths for all sound files.
        parse date and time info, sort, and return as a dataframe"""
    
    clipexts=['*.mp3']
    datefmt='%Y-%m-%d--%H.%M.mp3'

    # search through source folder for sound files
    # save clip path and date (parsed from filename)
    clippath = []
    clipdate = []
    for ext in clipexts:
        for root, dirnames, filenames in os.walk(srcdir):
            for filename in fnmatch.filter(filenames, ext):
                clippath.append(os.path.join(root, filename).replace('\\','/'))
                clipdate.append(datetime.strptime(filename, datefmt))
                
    # get sort index
    idx = np.argsort(clipdate)
    # retun sorted dataframe
    return pd.DataFrame({'date': np.array(clipdate)[idx], 
                       'path':  np.array(clippath)[idx]})   

In [86]:
import librosa
import librosa.display

import soundfile as sf

# dir containing sound files
srcdir = './data/every_other_hour'
destdir = './data/model/detections/'

df = read_clipnames(srcdir)

print('Scanning recording:')
# read and predict whale songs
for i, (dt, path) in df.iterrows():
    # load the recording audio, resample to clf's trained samprate
    y, sr = librosa.load(path, sr=clf.samprate)
    # predict whether any frames in this clip have whale song
    pred = clf.predict_from_wave(y)
    # save audio for any frames that model claims have whale song
    first = True
    for j,p in enumerate(pred):
        if p > 0:
            if first:
                print()
                print(os.path.basename(path).split('.')[0],end=' ')
                first = False
            print('%d,'%(j*clf.framesize/clf.samprate),end='')
            sf.write('%s_%03d.wav'%(destdir+os.path.basename(path).split('.')[0],
                    j*clf.framesize/clf.samprate), 
                    y[j*clf.framesize:(j+1)*clf.framesize], sr, subtype='PCM_16')
    

Scanning recording:

2015-01-01--04 56,64,68,76,156,192,
2015-01-02--08 244,
2015-01-02--20 60,68,76,88,96,104,120,140,164,172,180,192,200,208,
2015-01-02--22 0,4,8,12,16,20,24,28,32,36,40,44,48,52,56,60,92,
2015-01-03--00 0,4,8,12,16,20,24,28,32,36,40,44,48,52,56,60,64,68,72,76,80,84,88,92,96,104,108,112,116,120,124,128,132,136,140,144,152,160,168,172,176,180,184,188,192,196,200,204,208,212,220,224,228,240,248,256,260,268,276,280,284,288,292,
2015-01-05--00 60,
2015-01-05--02 0,164,
2015-01-05--16 172,
2015-01-05--22 164,
2015-01-07--10 56,
2015-01-07--12 156,
2015-01-08--10 4,20,52,56,64,72,76,80,88,92,96,104,112,120,132,140,148,152,156,160,168,172,176,180,184,188,192,256,296,
2015-01-08--12 4,16,20,28,32,48,52,56,60,64,68,76,84,88,92,96,100,104,108,116,124,132,140,144,148,152,156,160,164,168,172,176,184,192,200,204,208,212,216,220,224,228,236,244,248,256,260,264,272,280,284,292,
2015-01-08--14 80,88,92,100,104,108,112,120,124,128,132,140,144,152,160,164,284,288,
2015-01-08--16 72,10

2015-03-01--04 0,4,12,24,32,36,40,44,52,56,60,64,72,76,80,84,88,92,100,104,108,112,120,128,136,140,148,152,156,160,168,172,180,188,192,200,208,212,216,220,228,232,236,240,244,248,252,256,260,268,272,276,280,284,288,296,
2015-03-01--06 4,8,12,20,24,28,36,40,48,56,60,68,76,84,88,92,96,100,104,108,112,116,124,128,136,144,148,156,164,168,172,176,180,184,188,192,196,200,204,208,212,216,220,224,232,236,240,244,252,256,260,264,272,276,280,284,292,296,
2015-03-01--10 240,248,260,268,280,284,288,292,
2015-03-01--12 136,204,
2015-03-01--20 0,4,8,12,16,20,24,28,32,36,40,44,48,52,56,64,68,72,76,80,84,88,92,96,100,104,108,112,116,120,124,128,132,136,140,144,148,152,160,164,168,176,180,184,188,192,196,200,204,208,212,216,220,224,228,232,236,240,248,256,268,276,288,
2015-03-01--22 0,4,8,12,16,24,28,32,36,44,56,60,64,76,80,84,92,96,100,104,116,120,124,128,152,176,180,184,196,200,288,292,296,
2015-03-02--00 0,4,8,12,16,20,24,28,32,36,40,44,176,196,
2015-03-02--02 76,88,100,
2015-03-02--04 0,4,8,16,20,2

2015-04-01--04 0,20,32,92,132,144,172,184,192,196,204,212,216,224,236,244,248,288,
2015-04-01--08 72,164,232,
2015-04-01--10 60,92,100,292,
2015-04-01--12 48,52,172,
2015-04-01--16 288,292,
2015-04-01--20 236,268,
2015-04-01--22 96,116,208,
2015-04-02--08 256,264,268,276,280,288,
2015-04-02--14 76,124,136,164,
2015-04-02--18 4,8,16,20,28,36,48,56,76,84,96,124,144,152,164,172,192,200,212,220,224,232,240,260,
2015-04-02--20 156,164,184,188,196,200,212,224,232,244,252,256,264,276,288,
2015-04-03--00 24,48,60,96,176,
2015-04-03--02 0,12,24,36,48,60,84,96,108,120,140,152,196,232,244,256,264,288,
2015-04-03--04 84,220,240,
2015-04-03--06 96,200,228,256,268,276,284,296,
2015-04-03--08 0,4,8,12,16,20,24,28,32,36,40,44,52,56,60,64,72,76,84,96,100,116,120,140,152,160,196,204,280,296,
2015-04-03--10 8,16,20,24,28,32,40,44,48,52,56,60,64,72,76,80,84,88,92,96,100,104,108,112,116,124,128,132,136,140,144,148,152,156,160,164,168,172,176,180,184,188,192,196,200,204,208,212,216,220,224,232,236,244,252,2

2015-09-13--22 24,
2015-09-17--12 4,8,16,92,100,104,184,188,200,260,268,
2015-09-17--14 48,124,172,
2015-09-17--16 12,236,240,244,248,264,
2015-09-17--18 80,84,176,180,248,252,
2015-09-17--20 56,204,
2015-09-17--22 52,80,108,112,116,264,
2015-09-18--04 84,96,268,
2015-09-18--06 0,108,144,248,264,276,
2015-09-18--18 32,
2015-09-19--04 180,
2015-09-19--12 0,4,8,12,16,20,24,28,32,36,44,48,52,56,60,64,68,72,76,80,84,88,92,96,100,104,108,112,116,120,124,128,132,136,140,144,148,152,156,160,164,168,172,176,180,184,188,192,196,200,204,208,212,216,224,
2015-09-20--06 0,4,8,12,16,20,24,28,32,36,40,44,48,52,56,60,64,100,104,108,112,120,124,128,132,136,140,172,176,180,184,188,192,196,200,204,208,212,236,240,244,248,252,256,260,264,268,272,276,280,284,288,292,296,
2015-09-20--08 0,4,8,12,16,20,24,28,32,36,40,44,48,56,60,64,68,72,80,84,88,92,100,112,116,120,124,128,132,136,140,144,148,152,156,160,164,168,172,176,180,184,188,192,196,200,204,208,212,216,220,224,228,232,236,240,244,248,252,256,260,264,